In [1]:
import pandas as pd
data = pd.read_csv("结果4.csv")


In [3]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpInteger


# 筛选出SC60分拣中心的数据
sc60_data = data[data['分拣中心'] == 'SC60']

# 定义班次时间范围
shifts = {
    "00:00-08:00": range(0, 8),
    "05:00-13:00": range(5, 13),
    "08:00-16:00": range(8, 16),
    "12:00-20:00": range(12, 20),
    "14:00-22:00": range(14, 22),
    "16:00-24:00": range(16, 24)
}

# 按日期和班次整理每小时数据
shift_data = {}
for shift, hours in shifts.items():
    mask = sc60_data['小时'].isin(hours)
    grouped = sc60_data.loc[mask].groupby('日期')['货量'].sum()
    shift_data[shift] = grouped

# 初始化模型
model = LpProblem("Staff_Scheduling_SC60", LpMinimize)

# 定义变量
workers = range(1, 201)
dates = sc60_data['日期'].unique()
x = LpVariable.dicts("Regular", [(date, shift, worker) for date in dates for shift in shifts for worker in workers], 0, 1, LpBinary)
y = LpVariable.dicts("Temp", [(date, shift) for date in dates for shift in shifts], 0, None, LpInteger)

# 目标函数
model += lpSum(y.values())

# 添加约束
for shift, demands in shift_data.items():
    for date, demand in demands.items():
        model += (lpSum(25 * x[(date, shift, worker)] for worker in workers) + 20 * y[(date, shift)] >= demand, f"Cargo_Requirement_{date}_{shift}")

# 正式工出勤天数约束 
for worker in workers:
    model += lpSum(x[(date, shift, worker)] for date in dates for shift in shifts) <= 0.85 * len(dates), f"Max_Work_{worker}"

# 正式工每天只能安排一个班次
for worker in workers:
    for date in dates:
        model += lpSum(x[(date, shift, worker)] for shift in shifts) <= 1, f"One_Shift_Per_Day_{worker}_{date}"



In [4]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yooky/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ng/h1_4zcsj2qq895j6xpxvjyqh0000gn/T/8dcf075d1327455996aa9386116d54d5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ng/h1_4zcsj2qq895j6xpxvjyqh0000gn/T/8dcf075d1327455996aa9386116d54d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6385 COLUMNS
At line 187106 RHS
At line 193487 BOUNDS
At line 229668 ENDATA
Problem MODEL has 6380 rows, 36180 columns and 108180 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 197844 - 0.57 seconds
Cgl0008I 6000 inequality constraints converted to equality constraints
Cgl0003I 0 fixed, 180 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 6380 rows, 42180 columns (42180 integer (42000 of which binary))

KeyboardInterrupt: 